In [2]:
pip install wandb -q

Note: you may need to restart the kernel to use updated packages.


In [3]:
import wandb
wandb.init(project="", entity='')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: asy100 (hallymseonyeong). Use `wandb login --relogin` to force relogin


In [4]:
import torch 
import torchvision

from torchvision import datasets, models, transforms

import torch.nn as nn
import torch.optim as optim
import os

import numpy as np
import matplotlib.pyplot as plt

print(torch.__version__)

1.10.0


In [6]:
data_dir = "./hymenotera_data"

class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size
        
    def __call__(self, img):
        result = torch.reshape(img, self.new_size)
        return reuslt 
    
data_transforms = {
    'train' : transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
        
    ]),
    'val' : transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor()
        
    ]),
    
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

#load the entire dataset, we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'], batch_size=len(image_datasets['train']),
                                           shuffle=True)


In [7]:
class MyCNNModel(nn.Module):
    def __init__(self):
        super(MyCNNModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3*3*64, 1000)
        self.fc2 = nn.Linear(1000, 1)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        
        return out 
        
        
            
        

In [8]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.01


In [9]:
model = MyCNNModel()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(),  lr = LEARNING_RATE)

In [10]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc*100)
    
    return acc

In [15]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001

config = {"epochs": EPOCHS, "batch_size":BATCH_SIZE, "learning_rate":LEARNING_RATE}

In [16]:
wandb.init(project="", config=config)
#wandb.config.batch_size = BATCH_SIZE
#wandb.config.learning_rate = LEARNING_RATE
#config = {"epochs": EPOCHS, "batch_size":BATCH_SIZE, "learning_rate":LEARNING_RATE}

for e in range( 1, EPOCHS + 1 ):
    epoch_loss = 0
    epoch_acc =0 
    # data 를 GPU 로 전달해줌 
    for X_batch, y_batch in train_dataset:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    
    train_loss = epoch_loss/len(train_dataset)
    train_acc = epoch_loss/len(train_dataset)
    print(f'Epoch {e+0:03}: | Loss {epoch_loss/len(train_dataset):.5f} | Acc : {epoch_acc/len(train_dataset):.3f}')
    wandb.log({'accuracy': train_acc, 'loss': train_loss})

accuracy,▇▇▂█▁▃▃▃▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,▇▇▂█▁▃▃▃▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.53365
loss,0.53365


Epoch 001: | Loss 0.51555 | Acc : 73.000
Epoch 002: | Loss 0.51585 | Acc : 73.000
Epoch 003: | Loss 0.51078 | Acc : 74.000
Epoch 004: | Loss 0.52616 | Acc : 73.000
Epoch 005: | Loss 0.52173 | Acc : 72.000
Epoch 006: | Loss 0.51797 | Acc : 73.000
Epoch 007: | Loss 0.50319 | Acc : 73.000
Epoch 008: | Loss 0.48987 | Acc : 74.000
Epoch 009: | Loss 0.49790 | Acc : 73.000
Epoch 010: | Loss 0.48564 | Acc : 76.000
Epoch 011: | Loss 0.49939 | Acc : 74.000
Epoch 012: | Loss 0.48688 | Acc : 75.000
Epoch 013: | Loss 0.48474 | Acc : 73.000
Epoch 014: | Loss 0.48481 | Acc : 76.000
Epoch 015: | Loss 0.47053 | Acc : 75.000
Epoch 016: | Loss 0.47398 | Acc : 76.000
Epoch 017: | Loss 0.45140 | Acc : 76.000
Epoch 018: | Loss 0.48330 | Acc : 74.000
Epoch 019: | Loss 0.45066 | Acc : 76.000
Epoch 020: | Loss 0.46709 | Acc : 77.000
Epoch 021: | Loss 0.45908 | Acc : 76.000
Epoch 022: | Loss 0.45603 | Acc : 77.000
Epoch 023: | Loss 0.45336 | Acc : 78.000
Epoch 024: | Loss 0.44691 | Acc : 76.000
Epoch 025: | Los